Import Statements

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#To ensure all the plots are displayed in the notebook
%matplotlib inline 
import random
from pprint import pprint

Load Dataset and convert to pandas dataframe

In [2]:
#So we are formating the dataset to suit our api format which requires the class of flowers 
#to be called labels to signify that it is the output to training data points and also we
#converted data into dataframe for ease of operations with help of pandas
# from sklearn import datasets
# iris=datasets.load_iris()
# df= pd.DataFrame(data= np.c_[iris['data'], iris['target']],
#                  columns= iris['feature_names'] + ['target'])
# df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
# df=df.rename(columns={"species":"label"})
# df.drop("target",axis=1)

In [3]:
# We write our own train_test split function without using sklearn
def train_test_split(df,test_size):
    if isinstance(test_size,float):
        #if user inputs a proportion as test_size then convert to integer
        test_size=round(test_size*len(df))
    #convert the row indices to a vector for random func to use
    indices=df.index.tolist()
    #fetch random test indices to construct test_df
    test_indices=random.sample(population=indices,k=test_size)
    test_df=df.loc[test_indices]
    train_df=df.drop(test_indices)
    return train_df,test_df
#to check whether implementation is as intended
# random.seed(0)
# train_df,test_df=train_test_split(df,test_size=20)
# print(len(test_df),len(train_df))

Helper Functions

In [4]:
# We will operate on numpy arrays instead of pandas as numpy allows very fast operations

1) is Data pure?

In [5]:
#data should be a numpy 2-D array and this function returns a boolean
def isDataPure(data):
    label_col=data[:,-1]
    unique_classes=np.unique(label_col)
    if len(unique_classes)==1:
        return True
    else:
        return False
# test 
# isDataPure(train_df.values)
    

2) Classify data

In [6]:
# So we want to have a classify function which will be used in two cases if there 
# are no features left then it will return majority, else if we reach a pure class
def classifier(data):
    label_col=data[:,-1]
#   to fetch the unique classes we have in our data sets and the counts of their occurence
    unique_classes,unique_classes_counts=np.unique(label_col,return_counts=True)
    index_of_majority_class=unique_classes_counts.argmax()
    classification=unique_classes[index_of_majority_class]
    return classification
# test
# classifier(train_df[train_df["petal width (cm)"]<0.8].values)

3) Choose which feature to split on!

In [7]:
#Since we have continious dataset so we will have to choose a boundary to split on
#so for that we will write a function which will take in the data choose the best feature
#and returns a DICTIONARY, with key as the feature chosen and value as an array of 
#potential boundaries we can choose from.
def getPotentialSplits(data):
    potentialSplits={}
    _,nCols=data.shape
    #fetch all cols except last one(which is label column)
    for col in range(nCols-1):
        values=data[:,col]
        unique_values=np.unique(values)
        #this will return all the unique values in sorted order then around which we can
        #calculate the boundaries
        potentialSplits[col]=unique_values
    return potentialSplits
# potentialSplits=getPotentialSplits(train_df.values)

4) Split Data

In [8]:
#So we have to split data after selecting the feature on which we would split
#we have to pass in the feature and the value of the boundary we picked up from 
#potential splits and chose to split on. The function will return two 2-d numpy arrays
#with one representing all the values to the left of split boundary(smaller) and the other to the
#right(bigger)
def splitData(data,splitColumn,splitValue):
    splitColumnValues=data[:,splitColumn]
    type_of_feature=feature_types[splitColumn]
    if type_of_feature=="continuous":
        data_below=data[splitColumnValues<=splitValue]
        data_above=data[splitColumnValues>splitValue]
    else:
        data_below=data[splitColumnValues==splitValue]
        data_above=data[splitColumnValues!=splitValue]
        
    return data_below,data_above
# test
# splitCol=3
# splitVal=0.8
# data_below,data_above=splitData(train_df.values,splitCol,splitVal)

5) lowest overall entropy

In [9]:
def calculateEntropy(data):
    label_column=data[:,-1]
    _,counts=np.unique(label_column,return_counts=True)
    probabilities=counts/counts.sum()
    entropy=sum(probabilities* -np.log2(probabilities))
    return entropy
#test
# calculateEntropy(data_above)
def calculateOverallEntropy(data_below,data_above):
    num_data_points=len(data_below)+len(data_above)
    p_data_below=len(data_below)/num_data_points
    p_data_above=len(data_above)/num_data_points
    overallEntropy=(p_data_below*calculateEntropy(data_below)
                   +p_data_above*calculateEntropy(data_above))
    splitInfo=-((len(data_below)/num_data_points)*(np.log2(len(data_below)/num_data_points)))-((len(data_above)/num_data_points)*(np.log2(len(data_above)/num_data_points)))
    return overallEntropy,overallEntropy/splitInfo
#test
# calculateOverallEntropy(data_below,data_above)
# The function that calculates the best feature to split on by looking at entropies
# So this function calculates the entropy for all potential boundaries and selects
# the one with the lowest overall entropy which tells us that, that feature should
# be selected to split onx
def determineBestSplit(data,potentialSplits):
    overallEntropy=float('inf')
    gain_ratio=None
    for colIdx in potentialSplits:
        for value in potentialSplits[colIdx]:
            data_below,data_above=splitData(data,splitColumn=colIdx,splitValue=value)
            currentOverallEntropy,gain_r=calculateOverallEntropy(data_below,data_above)
            if currentOverallEntropy<=overallEntropy:
                overallEntropy=currentOverallEntropy
                bestSplitColumn=colIdx
                bestSplitValue=value
                gain_ratio=gain_r
    return bestSplitColumn,bestSplitValue,overallEntropy,gain_ratio
# test
# splitCol,splitVal=determineBestSplit(train_df.values,potentialSplits)

In [10]:
# Now we will classify a give input to the classes possible
def classify(inp,tree):
    question=list(tree.keys())[0]
    feature_name,comparison_operator,value=question.split()
    if comparison_operator=="<=":
        if inp[feature_name]<=float(value):
            answer=tree[question][0]
        else:
            answer=tree[question][1]
    else:
        if str(inp[feature_name])==value:
            answer=tree[question][0]
        else:
            answer=tree[question][1]
    #base case
    if not isinstance(answer,dict):
        return answer
    #recursive call
    else:
        return classify(inp,answer)
# we need a metric to test our model so we will write an accuracy function that will
# check our predicted value against original value and then take the mean of total predictions
# to calculate the accuracy
def accuracy(df,tree):
    df["classification"]=df.apply(classify,axis=1,args=(tree,))
    df["classification_correctness"]=df.classification==df.label
    accuracy=df.classification_correctness.mean()
    return accuracy
#This allows us to differentiate between categorical and continuous features
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

Decision Tree Algorithm

In [14]:
# So now we have all the ingredients to construct a decision tree algorithm
# to represent the tree we will use graph and the key of graph would be the feature
# and value of it can be either of the two things
# i) it's either a class which means we reached a leaf
# ii) or it can be a another feature on which we will split further
# so example of tree is
# tree={"petal width<=0.8":["Iris-setosa"],
#                            "petal width"<=1.65:[{"petal length<=4.9":["Iris-versicolor",
#                                                                      "Iris-virginica"]}],
#                                                 "Iris-virginica"]}]}
# {question:[yes,no]}--> Tree Representation
def decisionTree(df,counter=0,level=0):
    #so idea is to keep the api as simple as possible so we ask a df from user and covert
    #to numpy 2-d array and then pass it on recursively     
    #step1: data preparation
    if counter==0:
        global column_headers,feature_types
        column_headers=df.columns
        feature_types=determine_type_of_feature(df)
        data=df.values
    else:
        data=df
    #step2: base case (if data is pure or features are finished)
    if isDataPure(data):
        print("Reached Leaf Node")
        print("")
        classification=classifier(data)
        return classification
    #step3:recursive part(selecting features and splitting further)
    counter+=1
    #so this gets us all the potential splits and then determines the best feature 
    #to split on and then splits on basis of that feature.
    potentialSplits=getPotentialSplits(data)
    splitCol,splitVal,entropy,gain_ratio=determineBestSplit(data,potentialSplits)
    print("")
    print("Level ",level)
    print("Current Entropy is=",entropy)
    print("Splitting on feature:",column_headers[splitCol],"on value:",splitVal,"with gain ratio",gain_ratio)
    data_below,data_above=splitData(data,splitCol,splitVal)
    #check for empty data
    if len(data_below)==0 or len(data_above)==0:
        classification=classifier(data)
        return classification
    #so now we have to create subtrees for splitting further which will either result
    # in a leaf node or will form basis for further branching out of the tree.
    feature_name=column_headers[splitCol]
    type_of_feature=feature_types[splitCol]
    if type_of_feature=="continuous":
        question="{} <= {}".format(feature_name,splitVal)
    else:
        question="{} = {}".format(feature_name,splitVal)
    subTree={question:[]}
    #so to get the answers to our question we have to perform a split which will fill in
    # the answers. This is the recursive part of the algorithm
    yes=decisionTree(data_below,counter,level+1)
    no=decisionTree(data_above,counter,level+1)
    subTree[question].append(yes)
    subTree[question].append(no)
    return subTree
#test
# tree=decisionTree(train_df)
# pprint(tree)

In [16]:
df = pd.read_csv("converted_data.csv")
df = df.drop("Id", axis=1)
df = df.rename(columns={"species": "label"})
global dic
dic={}
i=0
for col in df.columns[:-1]:
    dic[i]=col
    i+=1
train_df, test_df = train_test_split(df, test_size=0.2)
tree = decisionTree(train_df)
pred_accuracy=accuracy(test_df, tree)
print("Accuracy is: ",pred_accuracy)


Level  0
Current Entropy is= 0.6749257854068225
Splitting on feature: petal_width on value: 0.5 with gain ratio 0.741891817518437
Reached Leaf Node


Level  1
Current Entropy is= 0.1670240204162191
Splitting on feature: petal_length on value: 4.8 with gain ratio 0.16704238631658044

Level  2
Current Entropy is= 0.05
Splitting on feature: petal_width on value: 1.6 with gain ratio 0.17458286045880028
Reached Leaf Node


Level  3
Current Entropy is= 0.0
Splitting on feature: sepal_width on value: 2.8 with gain ratio 0.0
Reached Leaf Node

Reached Leaf Node


Level  2
Current Entropy is= 0.08804001157162955
Splitting on feature: petal_width on value: 1.6 with gain ratio 0.16457808872265628

Level  3
Current Entropy is= 0.4
Splitting on feature: petal_width on value: 1.5 with gain ratio 0.4119674083156196
Reached Leaf Node


Level  4
Current Entropy is= 0.0
Splitting on feature: petal_length on value: 5.1 with gain ratio 0.0
Reached Leaf Node

Reached Leaf Node

Reached Leaf Node

Accuracy

C:\Users\Torre\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in log2
  from ipykernel import kernelapp as app
C:\Users\Torre\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app
